## 메타클래스
- 파이썬의 class문을 가로채서 클래스가 정의될 때마다 특별한 동작을 제공할 수 있게 해준다.

속성 접근을 동적으로 사용자화하기 위한 파이썬의 내장 기능 역시 강력한 도구다.
(Similarly mysterious and powerful are python’s built-in features for dynamically customizing attribute accesses.)
-> 파이썬의 객체 지향 구조와 함께 이용한다면 간단한 클래스를 쉽게 복잡한 클래스로 바꿔줄 수 있다.

단 과용할 경우엔 예상치못한 부작용을 불러올 수 있다.

## way 29. 게터와 세터 메서드 대신에 일반 속성을 사용하자
- 타 언어에서 파이썬으로 넘어온 프로그래머들은 자연스럽게 클래스에 게터와 세터를 구현하려고 한다. 
  하지만 이런 식으로 구현할 경우 즉석에서 증가시키기 같은 연산에는 사용하기 불편하며, 또한 파이썬에서는 명시적으로 구현할 일이 거의 없다. (*추정: 게터와 세터는 캡슐화하기 위한 메서드인데 파이썬에서는 명시적으로 데이터를 숨기지 않기 때문에 pythonic하지 않아서..?)

- 먼저 공개속성부터 구현해야한다.

In [2]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3

r1.ohms += 5e3  # 즉석에서 증가시키기 연산이 자연스러워졌다.

속성 설정시 특별한 동작이 일어나야 한다면, @property 데코레이터와 setter 속성을 사용할 수 있다. 여기서 @property는 게터 메소드 없이 프라이빗 속성의 값을 가져오는 데에 사용. setter 속성은 프라이빗 변수의 값을 세팅해줄 때 사용한다.

In [8]:
# voltage 프로퍼티 할당시 current 값이 바뀌도록 해본다.

class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

# 이제 voltage 프로퍼티에 할당하면 voltage 세터메서드가 실행되어 voltage에 맞게 current 프로퍼티를 업데이트할 것이다.        

r2 = VoltageResistance(1e3)
print(f'Before: {r2.current} amps')
r2.voltage = 10
print(f'Before: {r2.current} amps')

Before: 0 amps
Before: 0.01 amps


setter를 설정해 클래스에 전달된 값들의 타입을 체크하고 값을 검증할 수도 있다.

In [10]:
# 모든 저항값 > 0 보장

class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f"{ohms} ohms must be >0")
        self._ohms = ohms

In [12]:
r3 = BoundedResistance(1e3)
r3.ohms = 0  # 속성에 올바르지 않은 저항값을 할당

ValueError: 0 ohms must be >0

In [13]:
BoundedResistance(-5) # 생성자에 올바르지 않은 값 넘김
# BoundedResistance.__init__ -> Resistor.__init__ -> BoundedResistance 의 @ohms.setter

ValueError: -5 ohms must be >0

In [14]:
# 부모 속성을 불변으로 만드는 예제

class FixedResitance(Resistor):
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttrivuteError("Can;t set attribute")
        self._ohms = ohms

In [15]:
r4 = FixedResitance(1e3)
r4.ohms = 2e3

NameError: name 'AttrivuteError' is not defined

@property 메서드로 세터와 게터 구현시 예상과 다르게 동작하지 않게 해야한다.

## way 30. 속성을 리팩토링하는 대신 @property를 고려하자.
- @property 데코레이터를 이용하면 더 간결한 방식으로 인스턴스 속성에 접근할 수 있다. 
- 흔한 사용법중 하나로 단순 숫자 속성을 즉석에서 계산하는 방식으로 변경하는 것이 있다. 호출하는 쪽을 변경하지 않고도 기존에 슼ㄹ래스가 사용된 곳이 새 동작을 하게 해주므로 매우 유용한 기법이다.

In [17]:
# 구멍난 양동이
# 남은 할당량과 이 할당량을 이용할 수 있는 기간
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f"Bucket(quota={self.quota})"

# 양동이를 채울 때마다 할당량이 다음 기간으로 넘어가지 않게 동작
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

# 매번 사용할 양을 뺄 수 있는지 확인
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    elif bucket.quota < amount:
        return False
    bucket.quota -= amount
    return True

In [20]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Now enough for 99 quota")
    print(bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")
    print(bucket)

# 현재 양동이의 양을 알 수 없으므로 deduct 호출한 측에서 에러 발생시 무엇이 원인인지 알 수 없다.

Bucket(quota=100)
Had 99 quota
Not enough for 3 quota
Bucket(quota=1)


In [23]:
# max_quota, quota_consumed의 변경을 추적하도록 수정

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return f"Bucket(max_quota={self.max_quota}, quota_consumed={self.quota_consumed})"
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [25]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('fill', bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Now enough for 99 quota")

print('Now', bucket)
    
if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")

print('Still', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
fill Bucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now Bucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still Bucket(max_quota=100, quota_consumed=99)


- 예제에서 보다시피 Bucket.quota를 사용하는 코드는 변경할 필요가 없다.
- @property는 분명 유용한 도구이지만, 과용은 하지 말자. 과용하고 있다면, 클래스와 클래스를 사용하는 코드를 리팩토링해야하는 순간일 수도 있다.

## way 31. 재사용 가능한 @property 메소드에서는 디스크립터를 사용하자
- @property의 가장 큰 문제점은 재사용성이다. 관련없는 클래스에서는 사용이 불가능하다.

In [27]:
class Homework:
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")
        self._grade = value

galileo = Homework()
galileo.grade = 95

In [30]:
# 학생들의 시험성적 매기기

class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError("Grade must be between 0 and 100")

    # 시험 영역마다 새 @property와 관련 검증이 필요해서 장황해진다 
    @property
    def writing_grade(self):
        return self._writing_grade

    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value

    @property
    def math_grade(self):
        return self._math_grade

    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

# 범용으로 사용하기에 좋지 않다

### 디스크립터
- [《바인딩 동작》이 있는 객체 어트리뷰트로, 디스크립터 프로토콜의 메서드가 속성에 대한 접근을 재정의합니다. 메서드는 __get__(), __set__() 및 __delete__()가 있습니다.](https://docs.python.org/ko/3/howto/descriptor.html#id4)
-> 반복 코드 없이도 성적 검증 동작을 재사용할 수 있게 해준다!                                                                                                             

```
class Grade:
    def __get__(*args, **kwargs):
        # ...

    def __set__(*args, **kwargs):
        # ...


class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
```

~~~
exam = Exam()
exam.writing_grade = 40
~~~

-> Exam.__dict__['writing_grade'].__set__(exam, 40)

~~~
print(exam.writing_grade)
~~~

-> Exam.__dict__['writing_grade'].__get__(exam, 40)

Examp 인스턴스에 writing_grade 속성이 없으면 파이썬은 대신 해당 클래스의 속성을 이용한다. 이 클래스의 속성이 \__get__과 \__set__메서드를 갖춘 객체라면 파이썬은 디스크립터 프로토콜을 따른다고 가정한다.

In [33]:
# 잘못된 시도
class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError("시험 점수는 0 이상, 100 이하만 가능합니다")
        self._value = value

In [38]:
# 한 Exam 인스턴스에 있는 여러 속성에 접근하는 것은 기대한대로 동작
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99

print(f"Writing: {first_exam.writing_grade}, Science: {first_exam.science_grade}")

Writing: 82, Science: 99


In [40]:
# 여러 Exam 인스턴스의 이런 속성에 접근하면 이상 동작
second_exam = Exam()
second_exam.writing_grade = 75

print(f'Second {second_exam.writing_grade} is right')
print(f'First {first_exam.writing_grade} is wrong') # ..? 이거 왜...

Second 75 is right
First 82 is wrong


In [41]:
# 각 인스턴스별로 값을 추적해주기
class Grade:
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:
            return self

        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError("시험 점수는 0 이상, 100 이하만 가능합니다")
        self._values[instance] = value

In [43]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99

print(f"Writing: {first_exam.writing_grade}, Science: {first_exam.science_grade}")

second_exam = Exam()
second_exam.writing_grade = 75

print(f'Second {second_exam.writing_grade} is right')
print(f'First {first_exam.writing_grade} is right')

Writing: 82, Science: 99
Second 75 is right
First 82 is right


이 구현에서도 메모리 누수의 문제가 남아있다. _values 딕셔너리가 프로그램 수명 내내 모든 Exam 인스턴스의 참조를 저장하기 때문이다. 이 때 weakref를 사용하면 문제 해결이 가능하다.

In [44]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None:
            return self

        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError("시험 점수는 0 이상, 100 이하만 가능합니다")
        self._values[instance] = value

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99

print(f"Writing: {first_exam.writing_grade}, Science: {first_exam.science_grade}")

second_exam = Exam()
second_exam.writing_grade = 75

print(f'Second {second_exam.writing_grade} is right')
print(f'First {first_exam.writing_grade} is right')

Writing: 82, Science: 99
Second 75 is right
First 82 is right


## way 32 지연 속성에는 \_\_getattr\_\_, \_\_getattribute\_\_, \_\_setattr\_\_ 을 사용하자
- 파이썬의 언어 후크를 이용하면 시스템을 연계하는 범용 코드를 쉽게 만드는 것이 가능하다. 예를 들자면 데이터베이스와 객체를 연결하는 코드를 만들 때 로우와 스키마를 몰라도 개발이 가능하다.
- \_\_getattr\_\_ 라는 메서드가 이런 것이 가능하도록 한다. 클래스에 이 메서드를 정의하면 객체의 인스턴스 딕셔너리에서 속성을 찾을 수 없을 때마다 이 메서드가 호출된다

In [46]:
class LazyDB:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = 'value for ' + name
        setattr(self, name, value)
        return value

lazy = LazyDB()
print('Before:', lazy.__dict__)
print('foo   :', lazy.foo) # 존재하지 않는 속성 foo 접근시 __getattr__ 호출-> __dict__를 변경.
print('After :', lazy.__dict__)

Before: {'exists': 5}
foo   : value for foo
After : {'exists': 5, 'foo': 'value for foo'}


In [47]:
# __getattr__이 호출되는 지점을 보여주기 위해 로깅 추가한 예제
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('__getattr__ is called ')
        return super().__getattr__(name)

data = LoggingLazyDB()
print('Before:', data.__dict__)
print('foo   :', data.foo)
print('After :', data.__dict__) # 이후에는 foo가 저장되었으므로  __getattr__이 호출되지 않는다.

Before: {'exists': 5}
__getattr__ is called 
foo   : value for foo
After : {'exists': 5, 'foo': 'value for foo'}


데이터베이스 시스템에서 트랜잭션도 원한다고 하자. 사용자가 다음 번에 속성에 접근할 때는 대응하는 데이터베이스의 로우가 여전히 유효한지, 트랜잭션이 여전히 열려 있는지 알고 싶다고 하자. __getattr__ 후크는 기존 속성에 빠르게 접근하려고 객체의 인스턴스 딕셔너리를 사용하는 데 그치기 때문에 이 작업에는 믿고 쓸 수 없다.

이런 쓰임새를 위해서는 비슷하지만 다른 __getattribute__ 라는 또 다른 후크를 사용할 수 있다. 이 특별한 메소드는 객체의 속성에 접근할 때마다 호출되며, 심지어 해당 속성이 속성 딕셔너리에 있을 때도 호출된다. __getattr__ 메소드가 속성이 없을 때만 호출되는 것과는 대조적이다.

이런 동작 덕분에 속성에 접근할 때마다 전역 트랜잭션 상태를 확인하는 작업 등에 쓸 수 있다. 여기서는 이 메소드가 호출될 때마다 로그를 남기려고 ValidatingDB를 정의한다.

In [51]:
class ValidatingDB:
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print('__getattribute__ called with', name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'value for ' + str(name)
            setattr(self, name, value)
            return value

data = ValidatingDB()
print('Before:', data.__dict__)
print('foo   :', data.foo)
print('After :', data.__dict__)


# 동적으로 접근한 속성이 존재하지 않아야 하는 경우에는 AttributeError를 일으켜서 __getattr__, 
# __getattribute__에 속성이 없는 경우의 파이썬 표준 동작이 일어나게 한다.

__getattribute__ called with __dict__
Before: {'exists': 5}
__getattribute__ called with foo
foo   : value for foo
__getattribute__ called with __dict__
After : {'exists': 5, 'foo': 'value for foo'}


In [52]:
class MissingPropertyDB:
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError(name + " is not permitted here")

data = MissingPropertyDB()
data.bad_name

SyntaxError: invalid syntax (<ipython-input-52-fb5764236f07>, line 10)

파이썬 코드로 범용적인 기능을 구현할 때 종종 내장함수 \_\_hasattr\_\_ 로 프로퍼티가 있는지 확인하고 내장 함수 \_\_getattr\_\_ 로 프로퍼티 값을 가져온다. 이 함수들도 내부적으로 \_\_getattr\_\_을 호출하기 전에 인스턴스 딕셔너리에서 속성 이름을 찾는다.

In [53]:
data = LoggingLazyDB()

print('Before:', data.__dict__)
print('foo exists?', hasattr(data, 'foo'))
print('foo   :', data.foo)
print('After :', data.__dict__)
print('foo exists?', hasattr(data, 'foo'))

# 실행횟수 - __getattr__: 1, __getattribute__: 매번 실행

Before: {'exists': 5}
__getattr__ is called 
foo exists? True
foo   : value for foo
After : {'exists': 5, 'foo': 'value for foo'}
foo exists? True


이제 지연 방식으로 데이터를 데이터베이스에 집어넣고 싶다고 하자.

이 작업은 임의의 속성 할당을 가로채는 \_\_setattr\_\_ 언어 후크로 할 수 있다. \_\_getattr\_\_, \_\_getattribute\_\_로 속성을 추출하는 것과 다르게 별도의 매서드 두 개가 필요하지 않다.

__setattr__ 메소드는 인스턴스의 속성이 할당을 받을 때마다 직접 혹은, 내장 함수 setattr을 통해 호출된다.

In [56]:
class SavingDB:
    def __setattr__(self, name, value):
        super().__setattr__(name, value)

class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print(f'called __setattr__({name}, {value})')
        super().__setattr__(name, value)

data = LoggingSavingDB()
print('Before:', data.__dict__)
data.foo = 5
print('After :', data.__dict__)
data.foo = 7
print('Finally :', data.__dict__)

Before: {}
called __setattr__(foo, 5)
After : {'foo': 5}
called __setattr__(foo, 7)
Finally : {'foo': 7}


 \_\_getattribute\_\_과 \_\_setattr\_\_을 사용할 때 문제는 객체의 속성에 접근할 때마다 호출된다는 점이다.

In [58]:
# 객체의 속성에 접근하면 실제로 연관 딕셔너리에서 키를 찾게 하도록 바꿔보자
class BrokenDictionaryDB:
    def __init__(self):
        self._data = {}

    def __getattribute__(self, name):
        print('__getattribute__ called with', name)
        return self._data[name]
    
# 무한으로 재귀 발생
# __getattribute__ 가 self.dats에 접근하면 다시 __getattribute__가 실행되고 또 다시..
# 해결캑은 인스턴스에서 super().__getattribute__메서드로 인스턴스 속성 딕셔너리에서 값을 얻어오는 것

class DictionaryDB:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

# 마찬가지로 __setattr__에서도 super().__setattr__을 사용해아한다.

## way 33. 메타클래스로 서브클래스를 검증하자
- 메타클래스를 응용하는 가장 간단한 사례로 클래스를 올바르게 정의했는지 검증하는 것이 있다. 메타클래스는 서브클래스가 정의될 때마다 검증 코드를 실행하는 방법을 제공하므로 이럴 때 응용이 가능하다.
- 보통 클래스의 검증코드는 클래스의 객체 생성시 \_\_init\_\_ 메서드에서 실행된다. 메타클래스는 자체의 \_\_new\_\_ 메서드에서 연관된 class 문의 콘텐츠를 받는다. 여기서 타입이 실제로 생성되기 전에 클래스 정보를 수정할 수 있다.

In [63]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

print(MyClass.__dict__)

{'__module__': '__main__', 'stuff': 123, 'foo': <function MyClass.foo at 0x103b91840>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


클래스 정의전 클래스의 파라미터를 검증하려면 \_\_new\_\_ 메서드에 기능을 추가하면 된다.

In [65]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases != ():
            if class_dict['sides'] < 3:
                raise ValueError("Polygon needs 3+ sides")

        return type.__new__(meta, name, bases, class_dict)

class AbstractPolygon(metaclass=ValidatePolygon):
    sides = None

    @classmethod
    def sum_interior_angles(cls):
        return 180 * (cls.sides - 2)

class Triangle(AbstractPolygon):
    sides = 3

    
print('Before class')
class Line(AbstractPolygon):
    print('Before sides')
    sides = 1
    print('After sides')
print('After class')

Before class
Before sides
After sides


ValueError: Polygon needs 3+ sides

## Way 34 메타 클래스로 클래스의 존재를 등록하자

- 메타클래스 활용: 프로그램에 있는 타입을 자동으로 등록하기. <등록은 간단한 식별자를 대응하는 클래스에 매핑하는 역방향 조회를 수행할 때 유용하다.>?

In [66]:
# 파이썬 객체 -> JSON 으로 구현
import json

class Serializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({'args': self.args})

In [67]:
# Point2D -> 문자열로 쉽게 직렬화할 수 있다.
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'

point = Point2D(5, 3)
print('Point     :', point)
print('Serialized:', point.serialize())

Point     : Point2D(5, 3)
Serialized: {"args": [5, 3]}


In [78]:
# JSON 문자열을 역직렬화해서 JSON이 표현하는 Point2D 객체를 생성
# 간단한 불변 객체들을 범용적인 방식으로 직렬화하고 역직렬화가 가능하다
class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.dumps(json_data)
        return cls(params['args'])

# 문제점: 직렬화된 데이터에 대응하는 타입을 미리 알고 있을 때만 동작 가능 -> 좀 더 범용적으로 변경하기

In [74]:
class BetterSerializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__, # 직렬화할 객체의 클래스명을 포함하면 된다
            'args': self.args,
        })

In [80]:
# 클래스 명을 해당 클래스의 객체 생성자에 매핑하고 매핑 관리
registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class

def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

In [76]:
# 항상 제대로 동작하는것을 보장하기 위해 추후 역직렬화할 법한 모든 클래스에 registry_class 호출
class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

register_class(EvenBetterPoint2D)

이 방법은 만약 실수로 register_class 를 호출하는걸 깜박하면 등록을 잊은 클래스의 객체를 역직렬화하려 하므로, 코드가 중단되는 원인이 된다. 메타클래스를 이용하여 서브클래스가 정의될 때 class 문을 가로채는 방법으로 해결할 수 있다. 

In [81]:
class SerializeMeta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls

class RegisteredSerializable(BetterSerializable, metaclass=SerializeMeta):
    pass


class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return f'Vector 3D({self.x}, {self.y}, {self.z})'

v3 = Vector3D(10, -7, 100)
print('Before:', v3)
data = v3.serialize()
print('Serialized:', data)
after = deserialize(data)
print('After:', after)

Before: Vector 3D(10, -7, 100)
Serialized: {"class": "Vector3D", "args": [10, -7, 100]}
After: Vector 3D(10, -7, 100)


## way 35. 메타클래스로 클래스 속성에 주석을 달자.
- 메타클래스로 구현할 수 있는 기능중에 클래스 정의 이후 해당 클래스를 사용하기 전에 프로퍼티를 수정하거나 주석을 붙이는 것이 있다. 이 기법은 보통 디스크립터와 함께 사용한다.

In [82]:
# 고객 데이터베이스의 로우를 표현하는 새 클래스를 정의.
# 프로퍼티를 칼럼 이름과 연결하는데 사용할 디스크립터 클래스

class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

        
# 로우를 표현하는 클래스 정의
class Customer:
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')


foo = Customer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Parkito'
print('After :', repr(foo.first_name), foo.__dict__) # 중복 발생
# 클래스에서 클래스 변수로 디스크립터를 설정할 때, 각 변수의 이름을 두 번 적어야 하는 반복이 있다.

Before: '' {}
After : 'Parkito' {'_first_name': 'Parkito'}


- Field 생성자는 Field('first_name') 형태로 호출한다. 다음 이 호출의 반환 값이 Customer.field_name 에 할당된다. Field에서는 자신이 어떤 클래스 속성에 할당될지 미리 알 방법이 없다.
- 이런 중복성을 제거하려면 메타클래스를 사용하면 된다. class 문을 후킹하여 class 본문이 끝나자맞 원하는 동작을 가져올 수 있다.

In [83]:
# Field.name 과 Field.internal_name 을 디스크립터에 자동할당

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

필터 스크립터는 메타클래스를 사용한다고 해도 변경이 거의 없고 유일한 차이는 생성자에 인수를 넘길 필요가 없다는 점이다.